## speak onetime if person is apper

In [1]:
import cv2
import os
import time
import pyttsx3

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Define paths to the model files
prototxt_path = r'gender_pre_trained_models\gender_deploy.prototxt'
caffemodel_path = r'gender_pre_trained_models\gender_net.caffemodel'

# Check if files exist
if not os.path.isfile(prototxt_path):
    raise FileNotFoundError(f"Cannot find the prototxt file at {prototxt_path}")
if not os.path.isfile(caffemodel_path):
    raise FileNotFoundError(f"Cannot find the caffemodel file at {caffemodel_path}")

# Load the pre-trained Caffe models for gender classification
gender_net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

# Define the mean values for the gender model
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)

# Gender list
gender_list = ['Male', 'Female']

# Load the pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open the webcam
cap = cv2.VideoCapture(0)

# Initialize variables to keep track of detection and greeting logic
last_detect_time = 0
last_greet_time = 0
greeting_interval = 2  # Interval to check if person is not available
greeted = False

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) > 0:
        last_detect_time = time.time()
        if not greeted:
            for (x, y, w, h) in faces:
                # Extract the face region
                face_img = frame[y:y+h, x:x+w].copy()

                # Prepare the face image for classification
                blob = cv2.dnn.blobFromImage(face_img, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
                gender_net.setInput(blob)

                # Perform gender classification
                gender_preds = gender_net.forward()
                gender = gender_list[gender_preds[0].argmax()]

                # Draw a rectangle around the face and display the gender
                label = f'{gender}'
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
                cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

                # Say the greeting based on detected gender
                if gender == 'Male':
                    engine.say("Hello Sir")
                elif gender == 'Female':
                    engine.say("Hello Miss")
                engine.runAndWait()
                greeted = True
    else:
        # Check if the person has been missing for more than the greeting interval
        if time.time() - last_detect_time >= greeting_interval:
            greeted = False

    # Display the resulting frame
    cv2.imshow('Gender Classification', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


## good morning

In [2]:
from gtts import gTTS
import datetime
import pytz
import os
import time

def get_greeting():
    # Define the timezone for Sri Lanka
    sri_lanka_tz = pytz.timezone('Asia/Colombo')
    
    # Get the current time in Sri Lanka
    now = datetime.datetime.now(sri_lanka_tz)
    
    # Determine the greeting based on the current time
    hour = now.hour
    if 5 <= hour < 12:
        return "Good morning"
    elif 12 <= hour < 18:
        return "Good afternoon"
    else:
        return "Good evening"

def speak(text):
    # Convert text to speech
    tts = gTTS(text=text, lang='en')
    audio_file = 'greeting.mp3'
    tts.save(audio_file)
    
    # Play the audio file
    os.system(f"start {audio_file}")  # Use "start" for Windows, "open" for macOS, "xdg-open" for Linux

def main():
    # Define the interval for repeating greetings
    greeting_interval = 60 * 60  # 1 hour
    last_greeting_time = 0
    
    while True:
        current_time = time.time()
        
        # Check if it's time to speak a new greeting
        if current_time - last_greeting_time >= greeting_interval:
            greeting = get_greeting()
            speak(greeting)
            last_greeting_time = current_time
        
        # Sleep for a short time to avoid high CPU usage
        time.sleep(60)  # Check every minute

if __name__ == "__main__":
    main()


## combine code

In [1]:
import cv2
import os
import time
import pyttsx3
import datetime
import pytz

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Define paths to the model files
prototxt_path = r'gender_pre_trained_models\gender_deploy.prototxt'
caffemodel_path = r'gender_pre_trained_models\gender_net.caffemodel'

# Check if files exist
if not os.path.isfile(prototxt_path):
    raise FileNotFoundError(f"Cannot find the prototxt file at {prototxt_path}")
if not os.path.isfile(caffemodel_path):
    raise FileNotFoundError(f"Cannot find the caffemodel file at {caffemodel_path}")

# Load the pre-trained Caffe models for gender classification
gender_net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

# Define the mean values for the gender model
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)

# Gender list
gender_list = ['Male', 'Female']

# Load the pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open the webcam
cap = cv2.VideoCapture(0)

# Initialize variables to keep track of detection and greeting logic
last_detect_time = 0
last_greet_time = 0
greeting_interval = 2  # Interval to check if person is not available
greeted = False

def get_time_based_greeting():
    # Define the timezone for Sri Lanka
    sri_lanka_tz = pytz.timezone('Asia/Colombo')
    
    # Get the current time in Sri Lanka
    now = datetime.datetime.now(sri_lanka_tz)
    
    # Determine the greeting based on the current time
    hour = now.hour
    if 5 <= hour < 12:
        return "Good morning"
    elif 12 <= hour < 18:
        return "Good afternoon"
    else:
        return "Good evening"

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) > 0:
        last_detect_time = time.time()
        if not greeted:
            for (x, y, w, h) in faces:
                # Extract the face region
                face_img = frame[y:y+h, x:x+w].copy()

                # Prepare the face image for classification
                blob = cv2.dnn.blobFromImage(face_img, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
                gender_net.setInput(blob)

                # Perform gender classification
                gender_preds = gender_net.forward()
                gender = gender_list[gender_preds[0].argmax()]

                # Draw a rectangle around the face and display the gender
                label = f'{gender}'
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
                cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

                # Get the time-based greeting
                time_based_greeting = get_time_based_greeting()
                
                # Say the greeting based on detected gender
                if gender == 'Male':
                    engine.say(f"{time_based_greeting} Sir")
                elif gender == 'Female':
                    engine.say(f"{time_based_greeting} Miss")
                engine.runAndWait()
                greeted = True
    else:
        # Check if the person has been missing for more than the greeting interval
        if time.time() - last_detect_time >= greeting_interval:
            greeted = False

    # Display the resulting frame
    cv2.imshow('Gender Classification', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


## updated one

In [2]:
import cv2
import os
import time
import pyttsx3
import datetime
import pytz

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Define paths to the model files
prototxt_path = r'gender_pre_trained_models\gender_deploy.prototxt'
caffemodel_path = r'gender_pre_trained_models\gender_net.caffemodel'

# Check if files exist
if not os.path.isfile(prototxt_path):
    raise FileNotFoundError(f"Cannot find the prototxt file at {prototxt_path}")
if not os.path.isfile(caffemodel_path):
    raise FileNotFoundError(f"Cannot find the caffemodel file at {caffemodel_path}")

# Load the pre-trained Caffe models for gender classification
gender_net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

# Define the mean values for the gender model
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)

# Gender list
gender_list = ['Male', 'Female']

# Load the pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open the webcam
cap = cv2.VideoCapture(0)

# Initialize variables to keep track of detection and greeting logic
last_detect_time = 0
greeting_interval = 2  # Interval to check if person is not available
greeted = False

def get_time_based_greeting():
    # Define the timezone for Sri Lanka
    sri_lanka_tz = pytz.timezone('Asia/Colombo')
    
    # Get the current time in Sri Lanka
    now = datetime.datetime.now(sri_lanka_tz)
    
    # Determine the greeting based on the current time
    hour = now.hour
    if 5 <= hour < 12:
        return "Good morning"
    elif 12 <= hour < 18:
        return "Good afternoon"
    else:
        return "Good evening"

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) > 0:
        last_detect_time = time.time()

        if not greeted:
            for (x, y, w, h) in faces:
                # Extract the face region
                face_img = frame[y:y+h, x:x+w].copy()

                # Prepare the face image for classification
                blob = cv2.dnn.blobFromImage(face_img, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
                gender_net.setInput(blob)

                # Perform gender classification
                gender_preds = gender_net.forward()
                gender = gender_list[gender_preds[0].argmax()]

                # Draw a rectangle around the face and display the gender
                label = f'{gender}'
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
                cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

                # Get the time-based greeting
                time_based_greeting = get_time_based_greeting()
                
                # Say the greeting based on detected gender
                if gender == 'Male':
                    engine.say(f"{time_based_greeting} Sir")
                elif gender == 'Female':
                    engine.say(f"{time_based_greeting} Miss")
                engine.runAndWait()
                
                # Set greeted to True to prevent repeated greetings
                greeted = True
                break  # Exit the loop after greeting
    else:
        # Check if the person has been missing for more than the greeting interval
        if time.time() - last_detect_time >= greeting_interval:
            greeted = False

    # Display the resulting frame
    cv2.imshow('Gender Classification', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()
